### SE module


In [21]:
import math
import torch.nn as nn
import torch
import numpy

dim_in = 24
dim_mid = 24

kernel_reduce = torch.zeros(dim_mid, dim_in, 1, 1)
for k in range(dim_mid):
    for l in range(dim_in):
        for i in range(1):
            for j in range(1):
                kernel_reduce[k, l, i, j] = 0.1

kernel_expand = torch.zeros(dim_in, dim_mid, 1, 1)
for k in range(dim_in):
    for l in range(dim_mid):
        for i in range(1):
            for j in range(1):
                kernel_expand[k, l, i, j] = 0.005


# 實際上只做一次SE, make_divisible
# divisor=1
# 基本上這個東西沒做事(?)若se_ratio不為1，傳進來的v為小數，return new_v會處理成整數
# 目前看起來re_ratio!=se_ratio，則se_ratio為SE預設的0.25
def make_divisible(v, divisor=1):
    min_value = 1
    new_v = max(min_value, int(v + divisor / 2))
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        print('too small', new_v, v)
        new_v += divisor
    return new_v


def SE(dim_in, ratio, x):
    print('input data', x.shape)
    reduced_chs = int(dim_in * ratio)
    # self.conv = nn.Conv2d(dim_in, dim_out, kernel_size, stride, padding, dilation, groups, bias)
    conv_reduce = nn.Conv2d(dim_in, reduced_chs, 1, bias=False)
    #print('reduce kernel', conv_reduce.weight.data.shape)
    conv_reduce.weight.data = kernel_reduce
    conv_expand = nn.Conv2d(reduced_chs, dim_in, 1, bias=False)
    #print('expand kernel', conv_expand.weight.shape)
    conv_expand.weight.data = kernel_expand

    x_se = x.mean((2, 3), keepdim=True)  # (0, 1, 2, 3) => (b, c, h, w)
    print('x_se shape mean', x_se.shape)
    x_se = conv_reduce(x_se)
    print('x_se shape reduce', x_se.shape)
    x_se = torch.relu(x_se)
    x_se = conv_expand(x_se)

    return x * torch.sigmoid(x_se) 


In [22]:
import torch

batch_size = 1
dim_in = 24
image_size = 112
ratio = 1  # stage1-4: 4

# 生成隨機的模擬圖像數據，使用 PyTorch 的隨機函數
test_images = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                test_images[b, c, i, j] = (c+i+j) % 256

# 將數據傳遞給 SE 函數進行測試
x_se = SE(dim_in, ratio, test_images)

# 輸出測試結果
print(x_se.shape)
print(x_se)


input data torch.Size([1, 24, 112, 112])
x_se shape mean torch.Size([1, 24, 1, 1])
x_se shape reduce torch.Size([1, 24, 1, 1])
torch.Size([1, 24, 112, 112])
tensor([[[[  0.,   1.,   2.,  ..., 109., 110., 111.],
          [  1.,   2.,   3.,  ..., 110., 111., 112.],
          [  2.,   3.,   4.,  ..., 111., 112., 113.],
          ...,
          [109., 110., 111.,  ..., 218., 219., 220.],
          [110., 111., 112.,  ..., 219., 220., 221.],
          [111., 112., 113.,  ..., 220., 221., 222.]],

         [[  1.,   2.,   3.,  ..., 110., 111., 112.],
          [  2.,   3.,   4.,  ..., 111., 112., 113.],
          [  3.,   4.,   5.,  ..., 112., 113., 114.],
          ...,
          [110., 111., 112.,  ..., 219., 220., 221.],
          [111., 112., 113.,  ..., 220., 221., 222.],
          [112., 113., 114.,  ..., 221., 222., 223.]],

         [[  2.,   3.,   4.,  ..., 111., 112., 113.],
          [  3.,   4.,   5.,  ..., 112., 113., 114.],
          [  4.,   5.,   6.,  ..., 113., 114., 115.],